# Quantum Superdense Coding

### This is an implementation of superdense coding using quantum entanglement. The idea is that one party, say Alice, can send two bits of information to another party, say Bob, by transfering only one qubit. This works if both parties each had one qubit to begin with, which was part of an entangled state.

In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info import Statevector

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [2]:
# we first create the entangled state
qc = QuantumCircuit(2, 2)
qc.h(1)
qc.cx(1, 0)

initial_state = Statevector(qc)
initial_state.draw(output='latex')

<IPython.core.display.Latex object>

### Above we have the initial entangled Bell state. We give the left qubit to Alice and the right one to Bob.

In [3]:
# let Alice manipulate her qubit depending on the message she wants to send to Bob
def encode_message(qc, message):
    
    if len(message) != 2:
        raise Exception("Error - message of incorrect length has been passed!")
    if message[0] == '1':
        qc.z(1)
    if message[1] == '1':
        qc.x(1)

### Effectively, depending on the message Alice wants to send Bob, she applies different gates to her qubit. The options are:
 - ### 00 -> no gates applied
 - ### 01 -> X gate applied
 - ### 10 -> Z gate applied
 - ### 11 -> Z gate, followed by X gate, applied
 
### What these operations do are to transform the full entangled state into one of the four Bell states.

### Imagine Alice now sends her qubit from the entangled pair to Bob. He now has access to both qubits and can measure them. Note that Alice has only sent one qubit of information! Bob can deduce the message Alice wanted to send by measuring the two qubits, but first has to apply two gates:
- ### a controlled-NOT gate with his original qubit as the control, and the qubit received from Alice as the target;
- ### a Hadamard transform to his original qubit.

In [4]:
# decode message
def decode_message(qc):
    qc.cx(1, 0)
    qc.h(1)
    qc.measure([0, 1], [0, 1])
    sim = QasmSimulator()
    job = sim.run(qc)
    result = job.result()
    return result.get_counts()

In [8]:
# run the full program

# we first create the entangled state
qc = QuantumCircuit(2, 2)
qc.h(1)
qc.cx(1, 0)

message = '11'
encode_message(qc, message)
result = decode_message(qc)
print(result)

{'11': 1024}


### Effectively, all this works because we can generate all Bell states from the computational basis by applying a Hadmard transform to one of the two qubits, followed by a controlled-NOT gate with that qubit as the control.